In [1]:
import numpy as np
import pandas as pd
import emoji
from emoji.unicode_codes import UNICODE_EMOJI
import unicodedata
from unidecode import unidecode
from gensim import corpora
import gensim.models as gsm
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

C:\Users\mengy\Anaconda3\envs\py3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Load data
<pre>
t= pd.read_table('train.txt', names=['index', 'label','text'], header=None, delimiter="\t", quoting=3,encoding = "utf-8")
t.to_csv('train.csv', encoding='utf-8',index=False,header=True)

t17 = pd.read_table('SemEval2017-task4-test.subtask-A.english.txt', names=['index', 'label','text'], header=None, delimiter="\t", quoting=3,encoding = "utf-8")
t17.to_csv('2017test.csv', encoding='utf-8',index=False,header=True)

t16 = pd.read_table('twitter-2016test-A.txt', names=['index', 'label','text','nan'], header=None, delimiter="\t", quoting=3,encoding = "utf-8")
t16 = t16.drop('nan', axis=1)
t16.to_csv('2016test.csv', encoding='utf-8',index=False,header=True)

</pre>

I save the decoding data and reload then

In [2]:
t= pd.read_csv('train.csv')
t17 =  pd.read_csv('2017test.csv') 
t16 =  pd.read_csv('2016test.csv') 

### Pre-processing
take a example
<pre>
text = t17['text'][12275]
text
'✨✔😘😘Defeat The Ghost For Me! 👻 🔥🐠John Smith🔱🇨🇦:Zac Efron & beahttps://t.co/5kR4nbliE9 https://t.co/j8XdkHbi5A'
text_to_words1( text )
['defeat',
 'ghost',
 'john',
 'smith',
 'zac',
 'efron',
 'beaurl',
 'url',
 'heavi',
 'check',
 'mark',
 'face',
 'throw',
 'kiss',
 'face',
 'throw',
 'kiss',
 'ghost',
 'fire',
 'tropic',
 'fish',
 'trident',
 'emblem']
</pre>

In [3]:
emoji_pattern = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
def text_to_words( text ):
    #convert emoji into name and append at the end of text
    text = str(text)
    for char in text:
        if char in emoji.UNICODE_EMOJI:
            text = text + unicodedata.name(char) + " "
    #remove emojis from text
    text = re.sub(emoji_pattern, '', text)

    #remove any url to URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',text)
    #Convert any @Username to "AT_USER"
    text = re.sub('@[^\s]+','AT_USER',text)
    
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    text = re.sub('[\n]+', ' ', text)
    #Remove not alphanumeric symbols white spaces
    text = re.sub(r'[^\w]', ' ', text)
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')
    
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    tokens = word_tokenize(letters_only.lower())
    filtered = [word for word in tokens if word not in stop_words]
    stemmed = [stemmer.stem(word) for word in filtered] 
    
    return  stemmed

In [4]:
#doing training dataset
size = len(t['text'])
train = []
for i in range(size):
    #print states after 10000 reviews
    if ((i+1)%10000 == 0):
        print("Review %d of %d\n" % ( i+1, size ) )
    train.append( text_to_words( t['text'][i] ) )

Review 10000 of 23963

Review 20000 of 23963



In [28]:
#save cleaned data
output_train = pd.DataFrame({"index": t['index'],
                             "label":t['label'],
                      "text": train})
output_train.to_csv("clean_text_train.csv", index=False, header=True)

In [5]:
#doing test16 dataset
size = len(t16['text'])
test16 = []
for i in range(size):
    #print states after 10000 reviews
    if ((i+1)%10000 == 0):
        print("Review %d of %d\n" % ( i+1, size ) )
    test16.append( text_to_words( t16['text'][i] ) )


Review 10000 of 20633

Review 20000 of 20633



In [ ]:
output_test16 = pd.DataFrame({"index": t16['index'],
                             "label":t16['label'],
                      "text": test16})
output_test16.to_csv("clean_text_test16.csv", index=False, header=True)

In [6]:
#doing test17 dataset
size = len(t17['text'])
test17 = []
for i in range(size):
    #print states after 10000 reviews
    if ((i+1)%10000 == 0):
        print("Review %d of %d\n" % ( i+1, size ) )
    test17.append( text_to_words( t17['text'][i] ) )


Review 10000 of 12284



In [ ]:
output_test17 = pd.DataFrame({"index": t17['index'],
                             "label":t17['label'],
                      "text": test17})
output_test17.to_csv("clean_text_test17.csv", index=False, header=True)

### Feature Extraction
Now datasets are all cleaned, tokenized, and stemmed

#### Bags of words(Trivial one)

In [96]:
train_corpus = []
for text in train:
    train_corpus.append(" ".join( text))
    
test16_corpus = []
for text in test16:
    test16_corpus.append(" ".join( text))
    
test17_corpus = []
for text in test17:
    test17_corpus.append(" ".join( text))

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [98]:
#use the 3000 most frequent word only
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 3000)

train_bow = vectorizer.fit_transform(train_corpus)
train_bow = train_bow.toarray()

test16_bow = vectorizer.transform(test16_corpus)
test16_bow = test16_bow.toarray()

test17_bow = vectorizer.transform(test17_corpus)
test17_bow = test17_bow.toarray()

# re-weight the count features into floating point values suitable for usage by a classifier
transformer = TfidfTransformer(smooth_idf=False)

train_tfidf = transformer.fit_transform(train_bow)
train_bow_features1 = train_tfidf.toarray()    

test16_tfidf = transformer.transform(test16_bow)
test16_bow_features1 = test16_tfidf.toarray()

test17_tfidf = transformer.transform(test17_bow)
test17_bow_features1 = test17_tfidf.toarray()

Now I obtain my first features for train, test16, and test17.
They are train_bow_features1, test16_bow_features1, test17_bow_features1 

Without counting first, I use all cleaned data to get second feature

<pre>

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_bow = vectorizer.fit_transform(train_corpus)
train_bow = train_bow.toarray()

test16_bow = vectorizer.transform(test16_corpus)
test16_bow = test16_bow.toarray()

test17_bow = vectorizer.transform(test17_corpus)
test17_bow = test17_bow.toarray()

</pre>

I met memory error, so I didn't use this one

### Word Embedding

using Word2Vec twitter with dimension 400

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_twitter_model.bin', binary=True,unicode_errors='ignore')

In [134]:
def makeFeatureVec_w2v(words):

    featureVec = np.zeros((400,),dtype ='float32')    
    nwords = 0
    
    for word in words:
        if word in model.vocab:
            embedding_vector = model[word]
            if embedding_vector is not None: 
                nwords = nwords + 1
                featureVec = np.add(featureVec,embedding_vector)
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0:
        featureVec = np.divide(featureVec,nwords)
    return featureVec

In [138]:
w2v_feature_train = []
size = len(train)
for i in range(size):
    w2v_feature_train.append(makeFeatureVec_w2v(train[i]))

In [139]:
w2v_feature_train = np.asarray(w2v_feature_train)

In [148]:
w2v_feature_test16= []
size = len(test16)
for i in range(size):
    w2v_feature_test16.append(makeFeatureVec_w2v(test16[i]))

In [151]:
w2v_feature_test16 = np.asarray(w2v_feature_test16)

In [153]:
w2v_feature_test17= []
size = len(test17)
for i in range(size):
    w2v_feature_test17.append(makeFeatureVec_w2v(test17[i]))

In [154]:
w2v_feature_test17 = np.asarray(w2v_feature_test17)

using GloVe glove.twitter.27B.200d

In [11]:
from tqdm import tqdm 

In [12]:
embeddings_index = {}
f = open('glove.twitter.27B.200d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close()

1193514it [01:15, 15710.51it/s]


In [55]:
def makeFeatureVec(words):

    featureVec = np.zeros((200,),dtype ='float32')    
    nwords = 0
    # 
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,embedding_vector)
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0:
        featureVec = np.divide(featureVec,nwords)
    return featureVec

In [23]:
glove_feature_train = []

In [24]:
size = len(train)
for i in range(size):
    glove_feature_train.append(makeFeatureVec(train[i]))
    
    

In [26]:
glove_feature_train = np.asarray(glove_feature_train)

In [56]:
glove_feature_test16= []
size = len(test16)
for i in range(size):
    glove_feature_test16.append(makeFeatureVec(test16[i]))
 

In [57]:
glove_feature_test16 = np.asarray(glove_feature_test16)

In [58]:
glove_feature_test17= []
size = len(test17)
for i in range(size):
    glove_feature_test17.append(makeFeatureVec(test17[i]))

In [59]:
glove_feature_test17 = np.asarray(glove_feature_test17)

In [36]:
ytrain=np.zeros(len(train))
ytest16 = np.zeros(len(test16))
ytest17 = np.zeros(len(test17))

In [45]:
for i in range(len(train)):
    if t['label'][i] == 'negative':
        ytrain[i] = 0
    elif t['label'][i] == 'neutral':
        ytrain[i] = 1
    elif t['label'][i] == 'positive':
        ytrain[i] = 2

In [48]:
for i in range(len(test16)):
    if t16['label'][i] == 'negative':
        ytest16[i] = 0
    elif t16['label'][i] == 'neutral':
        ytest16[i] = 1
    elif t16['label'][i] == 'positive':
        ytest16[i] = 2

In [51]:
for i in range(len(test17)):
    if t17['label'][i] == 'negative':
        ytest17[i] = 0
    elif t17['label'][i] == 'neutral':
        ytest17[i] = 1
    elif t16['label'][i] == 'positive':
        ytest17[i] = 2

## Prediction

In [61]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


### Logistic

In [52]:
from sklearn import linear_model, datasets

GloVe word embedding

Predict 2017 test

c = 1

In [172]:
logreg_glove = linear_model.LogisticRegression(C=1).fit(glove_feature_train,ytrain)

In [173]:
log_glove_pre16 = logreg_glove.predict(glove_feature_test16)
print("F1 score is: ",f1_score(ytest16, log_glove_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, log_glove_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, log_glove_pre16, average="macro"))

F1 score is:  0.547744689637
precision score is:  0.570291041865
recall score is:  0.547254854431


In [174]:
log_glove_pre17 = logreg_glove.predict(glove_feature_test17)
print("F1 score is: ",f1_score(ytest17, log_glove_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, log_glove_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, log_glove_pre17, average="macro"))


F1 score is:  0.449391040891
precision score is:  0.489862971036
recall score is:  0.544328109444


c =10

In [77]:
logreg_glove = linear_model.LogisticRegression(C=10).fit(glove_feature_train,ytrain)

In [78]:
log_glove_pre16 = logreg_glove.predict(glove_feature_test16)
print("F1 score is: ",f1_score(ytest16, log_glove_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, log_glove_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, log_glove_pre16, average="macro"))

F1 score is:  0.548007772275
precision score is:  0.569747516317
recall score is:  0.547775013079


In [79]:
log_glove_pre17 = logreg_glove.predict(glove_feature_test17)
print("F1 score is: ",f1_score(ytest17, log_glove_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, log_glove_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, log_glove_pre17, average="macro"))


F1 score is:  0.448200037835
precision score is:  0.48831388166
recall score is:  0.543157523701


### SVC

In [80]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

c = 1

In [175]:
svc_glove = OneVsOneClassifier(LinearSVC(C=1)).fit(glove_feature_train,ytrain)

In [176]:
svc_glove_pre16 = svc_glove.predict(glove_feature_test16)
print("F1 score is: ",f1_score(ytest16, svc_glove_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, svc_glove_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, svc_glove_pre16, average="macro"))

F1 score is:  0.556289406102
precision score is:  0.570053330053
recall score is:  0.556032847424


In [177]:
svc_glove_pre17 = svc_glove.predict(glove_feature_test17)
print("F1 score is: ",f1_score(ytest17, svc_glove_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, svc_glove_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, svc_glove_pre17, average="macro"))

F1 score is:  0.458196656469
precision score is:  0.487107591202
recall score is:  0.546861783784


c =0.01

In [85]:
svc_glove = OneVsOneClassifier(LinearSVC(C=0.01)).fit(glove_feature_train,ytrain)

In [86]:
svc_glove_pre16 = svc_glove.predict(glove_feature_test16)
print("F1 score is: ",f1_score(ytest16, svc_glove_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, svc_glove_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, svc_glove_pre16, average="macro"))

F1 score is:  0.551254461045
precision score is:  0.576955384183
recall score is:  0.547893613295


In [87]:
svc_glove_pre17 = svc_glove.predict(glove_feature_test17)
print("F1 score is: ",f1_score(ytest17, svc_glove_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, svc_glove_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, svc_glove_pre17, average="macro"))

F1 score is:  0.45612917438
precision score is:  0.487830825354
recall score is:  0.544817749182


c = 100

In [88]:
svc_glove = OneVsOneClassifier(LinearSVC(C=100)).fit(glove_feature_train,ytrain)

In [89]:
svc_glove_pre16 = svc_glove.predict(glove_feature_test16)
print("F1 score is: ",f1_score(ytest16, svc_glove_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, svc_glove_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, svc_glove_pre16, average="macro"))

F1 score is:  0.493904495029
precision score is:  0.535331465036
recall score is:  0.519237192958


In [90]:
svc_glove_pre17 = svc_glove.predict(glove_feature_test17)
print("F1 score is: ",f1_score(ytest17, svc_glove_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, svc_glove_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, svc_glove_pre17, average="macro"))

F1 score is:  0.473071584131
precision score is:  0.467395308476
recall score is:  0.493054178442


## Naive Bayes

In [91]:
from sklearn.naive_bayes import GaussianNB

In [215]:
from sklearn.naive_bayes import MultinomialNB

In [216]:
nb_glove = GaussianNB().fit(glove_feature_train,ytrain)

In [94]:
nb_glove_pre16 = svc_glove.predict(glove_feature_test16)
print("F1 score is: ",f1_score(ytest16, nb_glove_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, nb_glove_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, nb_glove_pre16, average="macro"))

F1 score is:  0.493904495029
precision score is:  0.535331465036
recall score is:  0.519237192958


In [95]:
nb_glove_pre17 = svc_glove.predict(glove_feature_test17)
print("F1 score is: ",f1_score(ytest17, nb_glove_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, nb_glove_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, nb_glove_pre17, average="macro"))

F1 score is:  0.473071584131
precision score is:  0.467395308476
recall score is:  0.493054178442


## Word2Vec

### Logistic

In [157]:
logreg_w2v = linear_model.LogisticRegression(C=1).fit(w2v_feature_train,ytrain)

In [158]:
log_w2v_pre16 = logreg_w2v.predict(w2v_feature_test16)
print("F1 score is: ",f1_score(ytest16, log_w2v_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, log_w2v_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, log_w2v_pre16, average="macro"))

F1 score is:  0.561092233857
precision score is:  0.57874658414
recall score is:  0.558135880773


In [159]:
log_w2v_pre17 = logreg_w2v.predict(w2v_feature_test17)
print("F1 score is: ",f1_score(ytest17, log_w2v_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, log_w2v_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, log_w2v_pre17, average="macro"))

F1 score is:  0.460666060964
precision score is:  0.4960497611
recall score is:  0.554125898117


### SVC

In [162]:
svc_w2v = OneVsOneClassifier(LinearSVC(C=1)).fit(w2v_feature_train,ytrain)

In [163]:
svc_w2v_pre16 = svc_w2v.predict(w2v_feature_test16)
print("F1 score is: ",f1_score(ytest16, svc_w2v_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, svc_w2v_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, svc_w2v_pre16, average="macro"))

F1 score is:  0.567886471015
precision score is:  0.577526780096
recall score is:  0.566561096887


In [164]:
svc_w2v_pre17 = svc_w2v.predict(w2v_feature_test17)
print("F1 score is: ",f1_score(ytest17, svc_w2v_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, svc_w2v_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, svc_w2v_pre17, average="macro"))

F1 score is:  0.469403904932
precision score is:  0.493314174559
recall score is:  0.554983842779


### Naive Bayes

In [167]:
nb_w2v = GaussianNB().fit(w2v_feature_train,ytrain)

In [169]:
nb_w2v_pre16 = nb_w2v.predict(w2v_feature_test16)
print("F1 score is: ",f1_score(ytest16, nb_w2v_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, nb_w2v_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, nb_w2v_pre16, average="macro"))

F1 score is:  0.497824318389
precision score is:  0.510449342642
recall score is:  0.537545217423


In [171]:
nb_w2v_pre17 = nb_w2v.predict(w2v_feature_test17)
print("F1 score is: ",f1_score(ytest17, nb_w2v_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, nb_w2v_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, nb_w2v_pre17, average="macro"))

F1 score is:  0.466410262051
precision score is:  0.479074309766
recall score is:  0.479310439926


## Bags of Words

### Logistic

In [102]:
logreg_bow = linear_model.LogisticRegression(C=1).fit(train_bow_features1,ytrain)

In [103]:
log_bow_pre16 = logreg_bow.predict(test16_bow_features1)
print("F1 score is: ",f1_score(ytest16, log_bow_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, log_bow_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, log_bow_pre16, average="macro"))

F1 score is:  0.548461647842
precision score is:  0.58781725768
recall score is:  0.53658485928


In [104]:
log_bow_pre17 = logreg_bow.predict(test17_bow_features1)
print("F1 score is: ",f1_score(ytest17, log_bow_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, log_bow_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, log_bow_pre17, average="macro"))

F1 score is:  0.405522596993
precision score is:  0.48364451733
recall score is:  0.505526706393


### SVC

In [105]:
svc_bow = OneVsOneClassifier(LinearSVC(C=1)).fit(train_bow_features1,ytrain)

In [106]:
svc_bow_pre16 = svc_bow.predict(test16_bow_features1)
print("F1 score is: ",f1_score(ytest16, svc_bow_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, svc_bow_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, svc_bow_pre16, average="macro"))

F1 score is:  0.55524716356
precision score is:  0.558735763077
recall score is:  0.552638142701


In [107]:
svc_bow_pre17 = svc_bow.predict(test17_bow_features1)
print("F1 score is: ",f1_score(ytest17, svc_bow_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, svc_bow_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, svc_bow_pre17, average="macro"))

F1 score is:  0.419415977884
precision score is:  0.456890269012
recall score is:  0.501778551342


### Naive Bayes

In [108]:
nb_bow = GaussianNB().fit(train_bow_features1,ytrain)

In [109]:
nb_bow_pre16 = nb_bow.predict(test16_bow_features1)
print("F1 score is: ",f1_score(ytest16, nb_bow_pre16, average="macro"))
print("precision score is: ", precision_score(ytest16, nb_bow_pre16, average="macro"))
print("recall score is: ", recall_score(ytest16, nb_bow_pre16, average="macro"))

F1 score is:  0.240286324466
precision score is:  0.419766717532
recall score is:  0.379045652582


In [110]:
nb_bow_pre17 = nb_bow.predict(test17_bow_features1)
print("F1 score is: ",f1_score(ytest17, nb_bow_pre17, average="macro"))
print("precision score is: ", precision_score(ytest17, nb_bow_pre17, average="macro"))
print("recall score is: ", recall_score(ytest17, nb_bow_pre17, average="macro"))

F1 score is:  0.310988266061
precision score is:  0.397494022738
recall score is:  0.375633878463


## Majority Vote

In [179]:
ypre16 = []
ypre16.append(log_glove_pre16)
ypre16.append(svc_glove_pre16)
ypre16.append(nb_glove_pre16)
ypre16.append(log_w2v_pre16)
ypre16.append(svc_w2v_pre16)
ypre16.append(nb_w2v_pre16)
ypre16.append(log_bow_pre16)
ypre16.append(svc_bow_pre16)
ypre16.append(nb_bow_pre16)

In [180]:
ypre16 = np.asarray(ypre16)

In [195]:
majorityvote16 = np.zeros(ypre16.shape[1])

In [197]:
for i in range(ypre16.shape[1]):
    vote = np.zeros(3)
    for j in range(ypre16.shape[0]):
        if ypre16[j][i] == 0:
            vote[0] += 1
        elif ypre16[j][i] == 1:
            vote[1] += 1
        else:
            vote[2] +=1
    majorityvote16[i] = np.argmax(vote)
    

In [205]:
print("accuracy score is: ",accuracy_score(ytest16, majorityvote16))
print("F1 score is: ",f1_score(ytest16, majorityvote16, average="macro"))
print("precision score is: ", precision_score(ytest16, majorityvote16, average="macro"))
print("recall score is: ", recall_score(ytest16, majorityvote16, average="macro"))

accuracy score is:  0.616875878447
F1 score is:  0.587360808148
precision score is:  0.586767538674
recall score is:  0.588233053739


In [206]:
ypre17 = []
ypre17.append(log_glove_pre17)
ypre17.append(svc_glove_pre17)
ypre17.append(nb_glove_pre17)
ypre17.append(log_w2v_pre17)
ypre17.append(svc_w2v_pre17)
ypre17.append(nb_w2v_pre17)
ypre17.append(log_bow_pre17)
ypre17.append(svc_bow_pre17)
ypre17.append(nb_bow_pre17)

In [207]:
ypre17 = np.asarray(ypre17)

In [211]:
majorityvote17 = np.zeros(ypre17.shape[1])

In [212]:
for i in range(ypre17.shape[1]):
    vote = np.zeros(3)
    for j in range(ypre17.shape[0]):
        if ypre17[j][i] == 0:
            vote[0] += 1
        elif ypre17[j][i] == 1:
            vote[1] += 1
        else:
            vote[2] +=1
    majorityvote17[i] = np.argmax(vote)

In [213]:
print("accuracy score is: ",accuracy_score(ytest17, majorityvote17))
print("F1 score is: ",f1_score(ytest17, majorityvote17, average="macro"))
print("precision score is: ", precision_score(ytest17, majorityvote17, average="macro"))
print("recall score is: ", recall_score(ytest17, majorityvote17, average="macro"))

accuracy score is:  0.559182676653
F1 score is:  0.487125892223
precision score is:  0.500565656708
recall score is:  0.554786573987
